# 웹크롤러 만들기 - API의 활용¶

![banner](../image/4-1.jpg)

## API란 무엇일까요?

크롤링을 가장 편하게 활용할수 있는 방법은 API를 이용하는 방법이 있어요. API는 Application Programming Interface의 약자로 응용 프로그램에서 사용할 수 있도록, 운영 체제나 프로그래밍 언어가 제공하는 기능을 제어할 수 있게 만든 인터페이스를 말해요. 즉, 사용자가 어떤식으로 서버에 신호를 주면 어떤 내용이 반환되는지 정의되어 있습니다. 이를 이용하면 쉽고 간단하게 필요한 정보를 크롤링할 수 있어요.

MCP API Document(https://www.geniecontents.com/ko_doc/index.html)에서 제공하는 api를 활용해 보겠습니다. 이중 로또당첨번호 조회 api를 활용해 보도록 합시다.

![e-1](../image/e-1.jpg)

로또당첨번호 조회를 누르면 api의 사용법과 서버로 보낼 파라미터(parameter)에 대해 정의가 되어있습니다. 해당 api는 GET방식을 활용하도록 되어있네요.

![e-2](../image/e-2.jpg)

마우스를 더 내려보면 응답이 성공했을때 어떤 데이터를 받을수 있는지 정의되어 있습니다. 추첨번호 뿐만이 아니라 판매금액과 당첨금액까지 알수 있겠네요.

![e-3](../image/e-3.jpg)

자, 그럼 이것을 기반으로 본격적으로 프로그램을 만들어 봅시다.

In [1]:
# 필요 라이브러리 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json # 웹상에서 데이터는 일반적으로 사전형으로 생긴 json이라는 파일형태로 주고받습니다.

In [2]:
회차번호 = 1 # 예시로 첫번째 회차를 받아와 봅시다
url = "https://www.geniecontents.com/api/v1/lotto?drawNo={}".format(회차번호)

In [3]:
# 서버에 파라미터를 담아 요청(request)를 하면 적절한 응답(response)를 하게됩니다.
# 이때 위에 정의된 대로 get 방식으로 요청을 해야겠죠?
response = requests.get(url)

In [4]:
response.text

'{"resultCode":"R21000","statusCode":"200","body":{"drawDateYn":"N","num6":40,"bonusNum":16,"totalSellingPrice":3681782000,"num1":10,"num5":37,"drawNo":1,"drawDate":"2002-12-07","num4":33,"lottoResult":[{"sellingPriceByRank":0,"winningCnt":0,"rank":"FIRST","winningPriceByRank":0},{"sellingPriceByRank":143934100,"winningCnt":1,"rank":"SECOND","winningPriceByRank":143934100},{"sellingPriceByRank":143934000,"winningCnt":28,"rank":"THIRD","winningPriceByRank":5140500},{"sellingPriceByRank":287695800,"winningCnt":2537,"rank":"FOURTH","winningPriceByRank":113400},{"sellingPriceByRank":401550000,"winningCnt":40155,"rank":"FIFTH","winningPriceByRank":10000}],"num3":29,"num2":23}}'

됬습니다. 결과값이 성공적으로 왔군요! 이것을 보기 좋게 사전형(dict)로 바꿔봅시다. json 라이브러리를 활용하면 간단하게 문자열을 사전형 자료형으로 바꿀수 있어요.

In [5]:
response_json = json.loads(response.text)
response_json

{'resultCode': 'R21000',
 'statusCode': '200',
 'body': {'drawDateYn': 'N',
  'num6': 40,
  'bonusNum': 16,
  'totalSellingPrice': 3681782000,
  'num1': 10,
  'num5': 37,
  'drawNo': 1,
  'drawDate': '2002-12-07',
  'num4': 33,
  'lottoResult': [{'sellingPriceByRank': 0,
    'winningCnt': 0,
    'rank': 'FIRST',
    'winningPriceByRank': 0},
   {'sellingPriceByRank': 143934100,
    'winningCnt': 1,
    'rank': 'SECOND',
    'winningPriceByRank': 143934100},
   {'sellingPriceByRank': 143934000,
    'winningCnt': 28,
    'rank': 'THIRD',
    'winningPriceByRank': 5140500},
   {'sellingPriceByRank': 287695800,
    'winningCnt': 2537,
    'rank': 'FOURTH',
    'winningPriceByRank': 113400},
   {'sellingPriceByRank': 401550000,
    'winningCnt': 40155,
    'rank': 'FIFTH',
    'winningPriceByRank': 10000}],
  'num3': 29,
  'num2': 23}}

### 우승상금 구하기

우승상금은 'body'안에 'lottoResult' 안에 들어있군요. 가져와봅시다

In [6]:
우승상금 = response_json['body']['lottoResult']
우승상금

[{'sellingPriceByRank': 0,
  'winningCnt': 0,
  'rank': 'FIRST',
  'winningPriceByRank': 0},
 {'sellingPriceByRank': 143934100,
  'winningCnt': 1,
  'rank': 'SECOND',
  'winningPriceByRank': 143934100},
 {'sellingPriceByRank': 143934000,
  'winningCnt': 28,
  'rank': 'THIRD',
  'winningPriceByRank': 5140500},
 {'sellingPriceByRank': 287695800,
  'winningCnt': 2537,
  'rank': 'FOURTH',
  'winningPriceByRank': 113400},
 {'sellingPriceByRank': 401550000,
  'winningCnt': 40155,
  'rank': 'FIFTH',
  'winningPriceByRank': 10000}]

이것을 보기좋게 판다스의 데이터프레임을 이용해서 살펴볼까요?

In [7]:
pd.DataFrame(response_json['body']['lottoResult'])

,rank,sellingPriceByRank,winningCnt,winningPriceByRank
0,FIRST,0,0,0
1,SECOND,143934100,1,143934100
2,THIRD,143934000,28,5140500
3,FOURTH,287695800,2537,113400
4,FIFTH,401550000,40155,10000


### 당첨번호 구하기

당첨번호는 'body'안에 'num1'부터 'num6'까지에 들어있군요. 반복문을 이용해 가져와 볼까요?

In [8]:
당첨번호_리스트 = []
for i in range(1, 7):
    당첨번호_리스트.append(response_json['body']['num{}'.format(i)])
당첨번호_리스트

[10, 23, 29, 33, 37, 40]

비슷한 방식으로 보너스 번호도 가져와 봅시다.

In [9]:
보너스번호 = response_json['body']['bonusNum']
보너스번호

16

In [10]:
당첨번호_리스트.append(보너스번호)
당첨번호_리스트

[10, 23, 29, 33, 37, 40, 16]

이번에는 응용해서 1회부터 10회까지 모든 당첨번호(보너스번호 포함)를 가져와봅시다.

In [11]:
총_당첨번호_리스트 = []
for 회차번호 in range(1, 11):
    url = "https://www.geniecontents.com/api/v1/lotto?drawNo={}".format(회차번호)
    response = requests.get(url)
    response_json = json.loads(response.text)
    당첨번호_리스트 = []
    for i in range(1, 7):
        당첨번호_리스트.append(response_json['body']['num{}'.format(i)])
    보너스번호 = response_json['body']['bonusNum']
    당첨번호_리스트.append(보너스번호)
    
    총_당첨번호_리스트.append(당첨번호_리스트)

In [12]:
총_당첨번호_리스트

[[10, 23, 29, 33, 37, 40, 16],
 [9, 13, 21, 25, 32, 42, 2],
 [11, 16, 19, 21, 27, 31, 30],
 [14, 27, 30, 31, 40, 42, 2],
 [16, 24, 29, 40, 41, 42, 3],
 [14, 15, 26, 27, 40, 42, 34],
 [2, 9, 16, 25, 26, 40, 42],
 [8, 19, 25, 34, 37, 39, 9],
 [2, 4, 16, 17, 36, 39, 14],
 [9, 25, 30, 33, 41, 44, 6]]